In [0]:
import numpy as np
from scipy.optimize import linprog
def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To

    arc_idxs = [(arc[0], arc[1]) for arc in idxs]

    return NA, arc_idxs

# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

In [5]:
# IMPORT THE DATA:
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0]])
beq = np.array([0, 0, 0, 0, 0, 0])
C = np.array([0, 0, 0, 0, 0, 0, 0, -1])
upper_b = [7, 1, 2, 3, 2, 1, 2, None]
Aeq, arc_idxs = nn2na(NN)
#bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])
bounds = tuple([(0, upper_b[i]) for i in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n '
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

# OPTIMIZE: 
name_method='simplex'
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method=name_method)

# GET THE SOLUTION:
selarcs = get_selected_arcs(arc_idxs, res.x)
print('## Results ##')
print('Max flow: %s' % res.x[-1])

def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

usage = get_usage_string(arc_idxs, res.x.astype(int), upper_b)
min_cut = get_min_cut(arc_idxs, res.x, np.array(upper_b))
print('The usage of each arc will be (from, to): %s \n'
      'The arcs that produce the minimum cut (from, to): %s \n' % (usage, min_cut , ))

## Optimizer inputs ## 
Cost vector: [ 0  0  0  0  0  0  0 -1] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0 -1]
 [-1  0  1  1  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0]
 [ 0  0 -1  0  0  1  0  0]
 [ 0  0  0  0 -1  0  1  0]
 [ 0  0  0 -1  0 -1 -1  1]] 
b_eq demand-supply vector: [0 0 0 0 0 0] 
Bounds of each X arc variable: ((0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None)) 


## Results ##
Max flow: 5.0
The usage of each arc will be (from, to): {(0, 1): '4/7', (0, 2): '1/1', (1, 3): '1/2', (1, 5): '3/3', (2, 4): '1/2', (3, 5): '1/1', (4, 5): '1/2', (5, 0): '5/None'} 
The arcs that produce the minimum cut (from, to): [(0, 2), (1, 5), (3, 5)] 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
